# Affine Body Revolute Joint Limit Symbolics (atan2)

This notebook generates `affine_body_revolute_joint_limit.inl` using `atan2`-based `DeltaTheta`.

In [ ]:
import sys
import re
import pathlib as pl
import sympy as sp

# Keep the same style as other symbol notebooks.
sys.path.append("../")

def _find_repo_root() -> pl.Path:
    cwd = pl.Path.cwd().resolve()
    for candidate in [cwd, *cwd.parents]:
        if (candidate / "scripts").exists() and (candidate / "src").exists():
            return candidate
    raise RuntimeError("Cannot locate repository root from current working directory.")

repo_root = _find_repo_root()
scripts_dir = repo_root / "scripts"
sym_submodule = scripts_dir / "SymEigen" / "SymEigen.py"
if not sym_submodule.exists():
    raise FileNotFoundError(
        "SymEigen submodule is missing. Run: git submodule update --init --recursive scripts/SymEigen"
    )

sys.path.append(str(scripts_dir))
sys.path.append(str(scripts_dir / "SymEigen"))
sys.path.append(str(scripts_dir / "symbol_calculation"))

try:
    from SymEigen import *
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError(
        "Failed to import SymEigen. Ensure submodule is initialized: "
        "git submodule update --init --recursive scripts/SymEigen"
    ) from exc

from project_dir import backend_source_dir
from affine_body_core import compute_J_vec

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()


In [ ]:
# Two affine bodies i and j
qk = Eigen.Vector("qk", 12)
qk_t = Eigen.Vector("qk_t", 12)
ql = Eigen.Vector("ql", 12)
ql_t = Eigen.Vector("ql_t", 12)

def compute_cos_theta(bi, ni, bj, nj):
    return (bi.dot(bj) + ni.dot(nj)) / 2

def compute_sin_theta(bi, ni, bj, nj):
    return (ni.dot(bj) - bi.dot(nj)) / 2

def compute_revolute_delta_theta(F, F_t):
    bi = F[0:3, 0]
    ni = F[3:6, 0]
    bj = F[6:9, 0]
    nj = F[9:12, 0]

    bi_t = F_t[0:3, 0]
    ni_t = F_t[3:6, 0]
    bj_t = F_t[6:9, 0]
    nj_t = F_t[9:12, 0]

    cos_theta = compute_cos_theta(bi, ni, bj, nj)
    sin_theta = compute_sin_theta(bi, ni, bj, nj)
    cos_theta_t = compute_cos_theta(bi_t, ni_t, bj_t, nj_t)
    sin_theta_t = compute_sin_theta(bi_t, ni_t, bj_t, nj_t)

    num = sin_theta * cos_theta_t - cos_theta * sin_theta_t
    den = cos_theta * cos_theta_t + sin_theta * sin_theta_t
    return sp.atan2(num, den)

# Rest-shape space vectors: [b_bar, n_bar]
basis_k = Eigen.Vector("basis_k", 6)
basis_l = Eigen.Vector("basis_l", 6)
basis_m = Eigen.Vector("basis_m", 6)
basis_n = Eigen.Vector("basis_n", 6)


In [ ]:
def compute_J(basis_left, basis_right):
    Jr = sp.Matrix.zeros(12, 24)
    b_left = basis_left[0:3, 0]
    n_left = basis_left[3:6, 0]
    b_right = basis_right[0:3, 0]
    n_right = basis_right[3:6, 0]

    Jr[0:3, 0:12] = compute_J_vec(b_left)
    Jr[3:6, 0:12] = compute_J_vec(n_left)
    Jr[6:9, 12:24] = compute_J_vec(b_right)
    Jr[9:12, 12:24] = compute_J_vec(n_right)
    return Jr

Jr_i = compute_J(basis_k, basis_l)
Jr_j = compute_J(basis_m, basis_n)

Fr_q = Jr_i @ sp.Matrix.vstack(qk, ql)
Fr_q_t = Jr_i @ sp.Matrix.vstack(qk_t, ql_t)

delta_theta_q = compute_revolute_delta_theta(Fr_q, Fr_q_t)
d_delta_theta_dq = VecDiff(delta_theta_q, sp.Matrix.vstack(qk, ql))

G_F = Eigen.Vector("G_F", 12)
H_F = Eigen.Matrix("H_F", 12, 12)

JrT_G = Jr_i.T @ G_F
JrT_H_J = Jr_i.T @ H_F @ Jr_j

Cl_DeltaTheta = Gen.Closure(basis_k, qk, qk_t, basis_l, ql, ql_t)
Cl_F = Gen.Closure(basis_k, qk, basis_l, ql)
Cl_G = Gen.Closure(G_F, basis_k, basis_l)
Cl_H = Gen.Closure(H_F, basis_k, basis_l, basis_m, basis_n)

content = ""
content += f""" \n// Affine Body Revolute Joint Limit\n// Description: Symbolic expressions for affine body revolute joint limit.\n// Affine Body <-> Revolute Joint Frame Mapping\n\n// F = J_r * [qi; qj]\n{Cl_F("F", Fr_q)}\n// DeltaTheta\n{Cl_DeltaTheta("DeltaTheta", delta_theta_q)}\n// d(DeltaTheta)/dQ\n{Cl_DeltaTheta("dDeltaTheta_dQ", d_delta_theta_dq)}\n// JT_G = J_r^T * G_F\n{Cl_G("JT_G", JrT_G)}\n// JT_H_J = J_r^T * H_F * J_r\n{Cl_H("JT_H_J", JrT_H_J)}\n"""


In [ ]:
F = Eigen.Vector("F", 12)
F_t = Eigen.Vector("F_t", 12)
Cl = Gen.Closure(F, F_t)

delta_theta_f = compute_revolute_delta_theta(F, F_t)
d_delta_theta_df = VecDiff(delta_theta_f, F)
dd_delta_theta_ddf = VecDiff(d_delta_theta_df, F)

content += f"""\n// Theta and Derivatives for Affine Body Revolute Joint Limit\n\n{Cl("DeltaTheta", delta_theta_f)}\n{Cl("dDeltaTheta_dF", d_delta_theta_df)}\n{Cl("ddDeltaTheta_ddF", dd_delta_theta_ddf)}\n"""

if "atan2(" not in content:
    raise RuntimeError("Generated code does not contain atan2(...).")
if re.search(r"(?<!2)atan\(", content):
    raise RuntimeError("Legacy atan(...) detected in generated code.")

output_path = backend_source_dir("cuda") / "affine_body/constitutions/sym/affine_body_revolute_joint_limit.inl"
output_path.write_text(content, encoding="utf-8")
print("Written to:", output_path)
